In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
from typing import Dict
import os
from sklearn.preprocessing import LabelEncoder

def preprocess_image(image):
    try:
        if isinstance(image, str):
            img = Image.open(image)
        elif isinstance(image, np.ndarray):
            # BGR to RGB if from numpy/Gradio
            image = image[..., ::-1]
            img = Image.fromarray(image)
        else:
            return None, "Unsupported image format"
            
        # Always convert to RGB
        if img.mode != 'RGB':
            img = img.convert('RGB')
            
        # Resize first
        target_size = 256
        w, h = img.size
        ratio = min(target_size / w, target_size / h)
        new_w = int(w * ratio)
        new_h = int(h * ratio)
        resized = img.resize((new_w, new_h), Image.LANCZOS)
        
        # Calculate mean color from resized image
        resized_array = np.array(resized)
        mean_color = tuple(map(int, np.mean(resized_array, axis=(0, 1))))
        
        # Create padded image
        new_image = Image.new('RGB', (target_size, target_size), mean_color)
        
        # Calculate padding
        x_offset = (target_size - new_w) // 2
        y_offset = (target_size - new_h) // 2
        
        # Paste the resized image
        new_image.paste(resized, (x_offset, y_offset))
        
        # Convert to numpy and normalize
        final_array = np.array(new_image, dtype=np.float32) / 255.0
        
        return final_array, None
            
    except Exception as e:
        return None, f"Error processing image: {str(e)}"


class ArtStyleClassifier:
    def __init__(self, model_path: str = "art_classifier_curated.keras"):
        self.cnn_model = self._load_cnn_model(model_path)
        styles = [
            'naive_art', 'baroque', 'rococo', 'romanticism', 'art_deco',
            'american_realism', 'art_nouveau', 'expressionism', 'modernism',
            'post_impressionism', 'high_renaissance', 'cubism',
            'abstract_expressionism', 'art_informel', 'mannerism',
            'northern_renaissance', 'surrealism', 'symbolism',
            'early_renaissance', 'neo_romantic', 'ukiyo_e', 'impressionism',
            'pop_art', 'fauvism', 'neoclassicism', 'minimalism'
        ]
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(styles)
        self.style_labels = self.label_encoder.classes_

    def _load_cnn_model(self, model_path):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found: {model_path}")
        return tf.keras.models.load_model(model_path, compile=False)

    def preprocess_image(self, image):
        processed_image, error = preprocess_image(image)
        if error:
            raise Exception(error)
        return processed_image

    def predict(self, image):
        try:
            processed_image = self.preprocess_image(image)
            print(f"Processed image shape: {processed_image.shape}")
            print(f"Value range: {processed_image.min():.3f} to {processed_image.max():.3f}")
            
            # Check order consistency
            predictions = self.analyze_image(processed_image)
            probs_array = list(predictions.values())
            print(f"Max probability at index: {np.argmax(probs_array)}")
            print(f"Corresponds to label: {self.style_labels[np.argmax(probs_array)]}")
                
            # Sort predictions by probability
            sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
            
            # Format output
            output_text = "Top 5 Predictions:\n\n"
            for style, prob in sorted_predictions[:5]:
                output_text += f"{style}: {prob:.1%}\n"
            
            return output_text
        except Exception as e:
            return f"Error: {str(e)}"

    def analyze_image(self, image) -> Dict[str, float]:
        if len(image.shape) == 3:
            image = np.expand_dims(image, 0)
        
        probabilities = self.cnn_model.predict(image, verbose=0)[0]
        return {style: float(prob) for style, prob in zip(self.style_labels, probabilities)}

# Initialize the classifier
classifier = ArtStyleClassifier()

# Create Gradio interface
iface = gr.Interface(
    fn=classifier.predict,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Textbox(label="Predictions"),
    title="Art Style Classifier",
    description="Upload an artwork to classify its artistic style",
    
)

# Launch the interface
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 165, in __call__
    await sel

Processed image shape: (256, 256, 3)
Value range: 0.000 to 0.988
Max probability at index: 13
Corresponds to label: minimalism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 23
Corresponds to label: surrealism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 13
Corresponds to label: minimalism
Processed image shape: (256, 256, 3)
Value range: 0.043 to 1.000
Max probability at index: 13
Corresponds to label: minimalism
Processed image shape: (256, 256, 3)
Value range: 0.059 to 1.000
Max probability at index: 13
Corresponds to label: minimalism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 0
Corresponds to label: abstract_expressionism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 13
Corresponds to label: minimalism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 3
Corresp

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 165, in __call__
    await sel

Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 3
Corresponds to label: art_informel
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 19
Corresponds to label: pop_art
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 19
Corresponds to label: pop_art


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 165, in __call__
    await sel

Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 15
Corresponds to label: naive_art


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\Users\16148\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 165, in __call__
    await sel

Processed image shape: (256, 256, 3)
Value range: 0.008 to 1.000
Max probability at index: 12
Corresponds to label: mannerism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 1
Corresponds to label: american_realism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 1
Corresponds to label: american_realism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 23
Corresponds to label: surrealism
Processed image shape: (256, 256, 3)
Value range: 0.000 to 1.000
Max probability at index: 3
Corresponds to label: art_informel
